# SISTER workflow

In [38]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd
import IPython

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

### Create unique scene identifier

In [97]:
granules = ['https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20210730125208_20210730125212_0001.zip',
           'https://popo.jpl.nasa.gov/avng/y21/ang20210429t190537.tar.gz',
           'https://popo.jpl.nasa.gov/avcl/y06_data/f060925t01p00r06.tar.gz']

meta = 'SNOWTESTING'

scenes = []

crid = 998

for l1_granule in granules:
    
    landsat = 'None'   

    base_name = os.path.basename(l1_granule)

    if base_name.startswith('DESIS'):
        sensor = 'DESIS'
        datetime = base_name[31:46]

    elif base_name.startswith('PRS'):
        sensor = 'PRISMA'
        datetime = base_name[16:24] + 'T' + base_name[24:30]
        landsat='https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_%s_landsat.tar.gz' % base_name[16:50]

    elif base_name.startswith('ang'):
        sensor = 'AVNG'
        datetime = base_name[3:18]

    elif base_name.startswith('f'):
        sensor = 'AVCL'
        ''' AVIRIS classic filenames do not contain acquisition times,to be consistent with other
            sensors and to ensure identifier codes are unique a time string is created using other
            numbers in the filename            
        '''     

        datetime = "20%sT%s%s%s" % (base_name[1:7],
                                    base_name[8:10],
                                    base_name[11:13],
                                    base_name[14:16])
    else:
        raise ValueError('Unrecognized L1 datafile')

    job_args = {'sensor': sensor,
                'datetime': datetime,
                 'crid' : crid}
    job_args['preprocess'] = {'raw_dataset': l1_granule,
                              'landsat_dataset' : landsat}

    print(job_args)
    
    scenes.append(job_args)


{'sensor': 'PRISMA', 'datetime': '20210730T125208', 'crid': 998, 'preprocess': {'raw_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20210730125208_20210730125212_0001.zip', 'landsat_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_20210730125208_20210730125212_0001_landsat.tar.gz'}}
{'sensor': 'AVNG', 'datetime': '20210429t190537', 'crid': 998, 'preprocess': {'raw_dataset': 'https://popo.jpl.nasa.gov/avng/y21/ang20210429t190537.tar.gz', 'landsat_dataset': 'None'}}
{'sensor': 'AVCL', 'datetime': '20060925T010006', 'crid': 998, 'preprocess': {'raw_dataset': 'https://popo.jpl.nasa.gov/avcl/y06_data/f060925t01p00r06.tar.gz', 'landsat_dataset': 'None'}}


## Step 1. Preprocess

In [98]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        queue="sister-job_worker-32gb"
    else:
        queue="sister-job_worker-16gb"
    
    identifier = f'SISTER_{scene["sensor"]}_L1B_RDN_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_job_response = maap.submitJob(
        algo_id = "sister-preprocess",
        version = "sister-dev",
        raw_dataset = scene['preprocess']['raw_dataset'],
        landsat_dataset = scene['preprocess']['landsat_dataset'],
        crid = scene['crid'],
        publish_to_cmr = False,
        cmr_metadata={},
        queue=queue,
        identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {preprocess_job_response.status}')
    print(f'Job ID: {preprocess_job_response.id}')
          
    scenes[i]['preprocess']['job_id'] = preprocess_job_response.id
    

Identifier: SISTER_PRISMA_L1B_RDN_20210730T125208_998_SNOWTESTING
Submission status: success
Job ID: 625efa07-6986-4235-a4f4-9fd2b4f1de4f
Identifier: SISTER_AVNG_L1B_RDN_20210429t190537_998_SNOWTESTING
Submission status: success
Job ID: 9c08bed6-3435-4246-8013-c43f039ef211
Identifier: SISTER_AVCL_L1B_RDN_20060925T010006_998_SNOWTESTING
Submission status: success
Job ID: cf8ab819-be02-4d6f-bddd-f3ce13b8bf82


## Step 2. ISOFIT


In [89]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        segmentation_size = 100
    else:
        segmentation_size = 100
    
    identifier = f'SISTER_{scene["sensor"]}_L2A_RFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_id = scene['preprocess']['job_id'] 
    
    preprocess_result= [ x for x in maap.getJobResult(preprocess_id).outputs if x.startswith("s3://s3.") and "RDN" in x]
    preprocess_result.sort()
    l1b_rdn,l1b_loc, l1b_obs = preprocess_result
    
    scene['preprocess']['radiance_dataset'] =l1b_rdn
    scene['preprocess']['location_dataset'] =l1b_loc
    scene['preprocess']['observation_dataset'] =l1b_obs

    isofit_job_response = maap.submitJob(
                                    algo_id="sister-isofit",
                                    version="sister-dev",
                                    radiance_dataset=l1b_rdn,
                                    location_dataset = l1b_loc,
                                    observation_dataset = l1b_obs,
                                    segmentation_size = segmentation_size,
                                    n_cores=32,
                                    crid = scene['crid'],
                                    publish_to_cmr=False,
                                    cmr_metadata={},
                                    queue="sister-job_worker-32gb",
                                    identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {isofit_job_response.status}')
    print(f'Job ID: {isofit_job_response.id}')
    
    scene['isofit']  = {'job_id' : isofit_job_response.id}


## Step 3. Spectral resample

In [85]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_RSRFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    isofit_id = scene['isofit']['job_id'] 

    iso_result= [ x for x in maap.getJobResult(isofit_id).outputs if x.startswith("s3://s3.") and "RFL" in x]
    l2a_rfl,l2a_unc = iso_result
        
    scene['isofit']['reflectance_dataset'] =l2a_rfl
    scene['isofit']['uncertainty_dataset'] =l2a_unc

    resample_job_response = maap.submitJob(
                                            algo_id="sister-resample",
                                            version="sister-dev",
                                            reflectance_dataset= l2a_rfl,
                                            uncertainty_dataset= l2a_unc,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % resample_job_response.status)
    print('Job ID: %s' % resample_job_response.id)
    scene['resample']  = {'job_id' : resample_job_response.id}


Identifier: SISTER_AVNG_L2A_RSRFL_20170820t182018_999_TESTING
Submission status: success
Job ID: 93725a9f-c9e2-43bc-aa74-b4b1745d749c
Identifier: SISTER_AVCL_L2A_RSRFL_20130815T010007_999_TESTING
Submission status: success
Job ID: 08f4dd77-2828-4a28-9425-57dc0561f41d
Identifier: SISTER_PRISMA_L2A_RSRFL_20220409T185601_999_TESTING
Submission status: success
Job ID: 8d2e4f86-825b-4ac5-a955-3a73d18bfbdf
Identifier: SISTER_DESIS_L2A_RSRFL_20211120T035820_999_TESTING
Submission status: success
Job ID: 032ca4a1-c706-4299-9622-f54591147348


In [90]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_CORFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    resample_id = scene['resample']['job_id'] 
    resample_result= [ x for x in maap.getJobResult(resample_id).outputs if x.startswith("s3://s3.") and "RSRFL" in x]
    l2a_rsrfl,l2a_rsunc = resample_result
    
    scene['resample']['reflectance_dataset'] =l2a_rsrfl
    scene['resample']['uncertainty_dataset'] =l2a_rsunc



    rfl_corr_job_response = maap.submitJob(
                                            algo_id="sister-reflect_correct",
                                            version="sister-dev",
                                            observation_dataset= scene['preprocess']['observation_dataset'],
                                            reflectance_dataset= l2a_rsrfl,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % rfl_corr_job_response.status)
    print('Job ID: %s' % rfl_corr_job_response.id)
    scene['reflect_correct']  = {'job_id' : rfl_corr_job_response.id}


Identifier: SISTER_PRISMA_L2A_CORFL_20220409T185601_999_TESTING
Submission status: success
Job ID: dec00bb0-3f2f-4026-a044-ddc53d361436
Identifier: SISTER_DESIS_L2A_CORFL_20211120T035820_999_TESTING
Submission status: success
Job ID: 5a77f9bf-0862-4632-a26d-450dc0854c15


## Step 4. Fractional Cover

In [91]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_FRCOV_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    correct_id = scene['reflect_correct']['job_id'] 
    correct_result= [ x for x in maap.getJobResult(correct_id).outputs if x.startswith("s3://s3.") and "CORFL" in x]
    l2a_corfl =  correct_result[0]
    scene['reflect_correct']['reflectance_dataset'] = l2a_corfl

    frcover_job_response = maap.submitJob(
                                        algo_id="sister-fractional-cover",
                                        version="sister-dev",
                                        l2a_rfl=l2a_corfl,
                                        n_cores= 20,
                                        refl_nodata = -9998,
                                        refl_scale= 1,
                                        normalization = 'brightness',
                                        crid = scene['crid'],
                                        publish_to_cmr=False,
                                        cmr_metadata={},
                                        queue="sister-job_worker-32gb",
                                        identifier= identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % frcover_job_response.status)
    print('Job ID: %s' % frcover_job_response.id)
    scene['frcover']  = {'job_id' : frcover_job_response.id}

Identifier: SISTER_AVNG_L2B_FRCOV_20170820t182018_999_TESTING
Submission status: success
Job ID: 62b86363-bed0-459e-aabe-fca4de060f78
Identifier: SISTER_AVCL_L2B_FRCOV_20130815T010007_999_TESTING
Submission status: success
Job ID: f5a82cea-2441-47ea-a568-2134538decd5
Identifier: SISTER_PRISMA_L2B_FRCOV_20220409T185601_999_TESTING
Submission status: success
Job ID: a1764c1f-5960-4a7d-b5ea-0ccceb3854c1
Identifier: SISTER_DESIS_L2B_FRCOV_20211120T035820_999_TESTING
Submission status: success
Job ID: b51efbc7-2c3c-47e9-a161-f7444c15bf4a


## Step 6a. Vegetation biochemistry

In [94]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_VEGBIOCHEM_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    vegbiochem_job_response = maap.submitJob(
                            algo_id="sister-trait_estimate",
                            version="sister-dev",
                            reflectance_dataset= scene['reflect_correct']['reflectance_dataset'],
                            frcov_dataset=l2b_frcov,
                            veg_cover = 0.5,
                            crid = scene['crid'],
                            publish_to_cmr=False,
                            cmr_metadata={},
                            queue="sister-job_worker-16gb",
                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % vegbiochem_job_response.status)
    print('Job ID: %s' % vegbiochem_job_response.id)
    scene['vegbiochem']  = {'job_id' : vegbiochem_job_response.id}

Identifier: SISTER_AVNG_L2B_VEGBIOCHEM_20170820t182018_999_TESTING
Submission status: success
Job ID: 94e23f01-c4d2-4a6d-b94e-7ffb94392255
Identifier: SISTER_AVCL_L2B_VEGBIOCHEM_20130815T010007_999_TESTING
Submission status: success
Job ID: 79b7d17f-3c81-4c69-bb03-eab1c30ef66e
Identifier: SISTER_PRISMA_L2B_VEGBIOCHEM_20220409T185601_999_TESTING
Submission status: success
Job ID: 61157bcd-cfb8-4a4c-822a-f2a877ef7862
Identifier: SISTER_DESIS_L2B_VEGBIOCHEM_20211120T035820_999_TESTING
Submission status: success
Job ID: e2b4af88-4877-420c-91f6-162132bc600a


## Step 6b. Snow grainsize

In [140]:
grainsize_job_response = maap.submitJob(
    algo_id="sister-grainsize",
    version="sister-dev",
    l2a_rfl=l2a_corfl,
    l2b_frcov=l2b_frcov,
    snow_cover = 0.9,
    CRID= '123',
    publish_to_cmr=False,
    cmr_metadata={},
    queue="sister-job_worker-32gb",
    identifier="%s_L2B_SNOWTEST" % identifier)

print('Submission status: %s' % grainsize_job_response.status)
print('Job ID: %s' % grainsize_job_response.id)

Submission status: success
Job ID: cc97aa3b-bca6-45bc-8cc5-22b11d52703f
